# Action Provider Demo: send requests directly to an AP endpoint

### 1.1 Select an Action Provider Host

In [ ]:
"""Use Diaspora AP directly through the access endpoint."""

from __future__ import annotations

import json

import requests
from diaspora_event_sdk import Client as GlobusClient

# ap_endpoint = 'http://127.0.0.1:8000/'
ap_endpoint = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com'
print('AP endpoint selected:', ap_endpoint)

### 1.2 Retrieve Credential

In [ ]:
"""A hacky way to retrieve access token."""  # noqa: Q001
c = GlobusClient()
# c.logout()
# c = GlobusClient()
print("User's OpenID:", c.subject_openid)
tokens = c.login_manager._token_storage.get_token_data(
    '2b9d2f5c-fa32-45b5-875b-b24cd343b917',
)
access_token = tokens['access_token']
print("User's access token:", access_token)

### 1.3 Select a Topic

In [ ]:
topic = 'topic' + c.subject_openid[-12:]
print(c.register_topic(topic))
print(c.list_topics())
print('Topic to produce/consume:', topic)

### 1.4 Set request header

In [ ]:
headers = {
    'authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

### 2.1.1 Produce Messages to AP without msg keys

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.1.2: Error Case: `msgs` does not exist

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.1.3: Error Case: `msgs` is empty

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.1.4: Error Case: the topic does not exist (takes 10 seconds)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': '__badtopic',
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.2.1 Produce Messages to AP with a Single Key

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': 'my-key',
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.3.1 Produce Messages to AP with a List of Keys

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1', 'my-key2', 'my-key3'],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.3.2 Error case: the length of `keys` does not match the length of `msgs`

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1'],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.1: Consume All Messages (no `ts` nor `group_id`)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.2: Consume Messages since a Timestamp (has `ts` but no `group_id`)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1717532033372,
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.3: Error case: the topic does not exist

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': '__badtopic',
        'ts': 1717532033372,
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.4: Error case: the user does not have access 

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': 'diaspora-cicd',
        'ts': 1715930522000,
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.2.1 Consume with a Prefix Filter

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1717532033372,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.2.2 Consume with a Filter that has Multiple Conditions (cond1 AND cond2)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1717532033372,
        'filters': [
            {
                'Pattern': {
                    'value': {
                        'content': [
                            {'prefix': 'hello', 'suffix': 'world2'},
                        ],
                    },
                },
            },
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.2.3 Consume with Multiple Filters (filter1 OR filter2)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1717532033372,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'hello world2'}]}}},
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.2.3 Consume with Multiple Filters (repeated filters do not return repeated msgs)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1717532033372,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'hello world1'}]}}},
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.2.4 Error case: Consume with a Bad Filter

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1715930522000,
        'filters': [
            {
                'BadPattern': {
                    'value': {'content': [{'prefix': 'hello world1'}]},
                },
            },
            {'Pattern': {'value': {'content': [{'suffix': 'hello world1'}]}}},
        ],
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.3.1 Consume with a `group_id` but no `ts` provided

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.3.2 The same request not return new msgs under the same `group_id` since the last one was successful

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.3.3 Consume with a `group_id` and a `ts` (all retrieved msgs >= `ts`, if there's any)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'group_id': 'my-group12',
        'ts': 1715930522000,
    },
}
response = requests.post(f'{ap_endpoint}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 4.1.1 Check Request Status, Cancel Request, and Release Request (Not Implemented and Always Succeed)

In [ ]:
action_id = '1mX5FKSJTFMy'

response = requests.get(
    f'{ap_endpoint}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)


for endpoint in ['cancel', 'release']:
    response = requests.post(
        f'{ap_endpoint}/{action_id}/{endpoint}',
        headers={'authorization': f'Bearer {access_token}'},
    )

    # Print the response from the server
    print('Response status code:', response.status_code)
    response_content = response.content.decode('utf-8')
    parsed_content = json.loads(response_content)
    formatted_content = json.dumps(parsed_content, indent=2)
    print('Response content:', formatted_content)